# Imports

In [5]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

Function to scrape data from 'www.wunderground.com'. Takes airport's IATA and ICAO code as arguments and returns a DataFrame.

In [26]:
def scrape_2015_weather_by_airport(airport,code):
    page = requests.get("https://www.wunderground.com/history/airport/"+code+"/2015/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=2015&req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=")
    soup = BeautifulSoup(page.content, 'html.parser')
    table=soup.find('table',class_="responsive obs-table daily")
    c_month=0
    c_day=1
    month=[]
    day=[]
    temperature=[]
    vis=[]
    rain=[]
    wind=[]
    port=[]
    Fog=[]
    Hail=[]
    Snow=[]
    Thunderstorm=[]
    Tornado=[]
    events=['Fog', 'Hail', 'Rain', 'Snow', 'Thunderstorm', 'Tornado']
    tbody = table.find_all('tbody')
    for i in range(len(tbody)):
        trs=tbody[i].find_all('tr')    
        if(trs[0].find('td',class_='daily-td-grey')):
            c_month=c_month+1
            c_day=1
        else:
            tds=trs[0].find_all('td')
            d=tds[0].find_all('a')
            d=d[0].text
            temp=tds[2].find_all('span')
            if(len(temp)>0):
                temperature.append(temp[0].text)
            else:
                temperature.append(np.nan)

            temp=tds[14].find_all('span')
            if(len(temp)>0):
                vis.append(temp[0].text)
            else:
                vis.append(np.nan)

            temp=tds[17].find_all('span')
            if(len(temp)>0):
                wind.append(temp[0].text)
            else:
                wind.append(np.nan)

            temp=tds[19].find_all('span')
            if(len(temp)>0):
                if(temp[0].text=='T'):
                    rain.append(0)
                else:
                    rain.append(temp[0].text)
            else:
                rain.append(np.nan)

            temp=tds[20]
            if(1==1):
                temp=temp.text
                temp=temp.replace('\n','')
                temp=temp.replace('\t','')
                e=temp.split(',')
                f=h=r=t=to=s=0
                for event in events:
                    if (event=='Fog' and e.count(event)>0):
                        f=1
                    if (event=='Hail' and e.count(event)>0):
                        h=1
                    if (event=='Snow' and e.count(event)>0):
                        s=1
                    if (event=='Thunderstorm' and e.count(event)>0):
                        t=1 
                    if (event=='Tornado' and e.count(event)>0):
                        to=1
                    
                Fog.append(f)
                Hail.append(h)
                Snow.append(s)
                Thunderstorm.append(t)
                Tornado.append(to)
            
            c_day=d
            month.append(c_month)
            day.append(c_day)
            port.append(airport)
            
    df=pd.DataFrame()
    df['Month']=month
    df['Day']=day
    df['Airport']=port
    df['Temperature']=temperature
    df['Visibility (km)']=vis
    df['Wind (km/h)']=wind
    df['Precip. (mm)']=rain
    df['Fog']=Fog
    df['Hail']=Hail
    df['Snow']=Snow
    df['Thunderstorm']=Thunderstorm
    df['Tornado']=Tornado

    return df

# Steps:
1. Read 'airport.csv' and 'icao.csv'
2. Find ICAO code from 'icao.csv' corresponding to IATA codes in 'airports.csv'
3. Call 'scrape_2015_weather_by_airport(iata,icao)'
4. Append all data to 'weather_data'

In [7]:
%%time
airports=pd.read_csv('airports.csv')
port=pd.read_csv('icao.csv',dtype ={'ident':str})
port=port[port['iso_country']=='US']
weather_data=pd.DataFrame()
for i in range(len(airports)):
    print('scraping :'+airports['IATA_CODE'][i])
    weather_data=weather_data.append(scrape_2015_weather_by_airport(airports['IATA_CODE'][i],port.loc[port['iata_code'] == airports['IATA_CODE'][i]]['ident'].values[0]))
weather_data.to_csv('weather2015_final.csv')

scraping :ABE
scraping :ABI
scraping :ABQ
scraping :ABR
scraping :ABY
scraping :ACK
scraping :ACT
scraping :ACV
scraping :ACY
scraping :ADK
scraping :ADQ
scraping :AEX
scraping :AGS
scraping :AKN
scraping :ALB
scraping :ALO
scraping :AMA
scraping :ANC
scraping :APN
scraping :ASE
scraping :ATL
scraping :ATW
scraping :AUS
scraping :AVL
scraping :AVP
scraping :AZO
scraping :BDL
scraping :BET
scraping :BFL
scraping :BGM
scraping :BGR
scraping :BHM
scraping :BIL
scraping :BIS
scraping :BJI
scraping :BLI
scraping :BMI
scraping :BNA
scraping :BOI
scraping :BOS
scraping :BPT
scraping :BQK
scraping :BQN
scraping :BRD
scraping :BRO
scraping :BRW
scraping :BTM
scraping :BTR
scraping :BTV
scraping :BUF
scraping :BUR
scraping :BWI
scraping :BZN
scraping :CAE
scraping :CAK
scraping :CDC
scraping :CDV
scraping :CEC
scraping :CHA
scraping :CHO
scraping :CHS
scraping :CID
scraping :CIU
scraping :CLD
scraping :CLE
scraping :CLL
scraping :CLT
scraping :CMH
scraping :CMI
scraping :CMX
scraping :CNY
scrapi

# Checking missing values

In [8]:
weather_data=pd.read_csv('weather2015_final.csv')
weather_data.isnull().sum()

Unnamed: 0          0
Month               0
Day                 0
Airport             0
Temperature        23
Visibility (km)    39
Wind (km/h)        19
Precip. (mm)        1
Fog                 0
Hail                0
Snow                0
Thunderstorm        0
Tornado             0
dtype: int64

# Handeling missing values

The missing values are replaced by the average of previous and next three days values

In [9]:
cols_with_nulls=['Temperature','Visibility (km)','Wind (km/h)','Precip. (mm)']
for col in cols_with_nulls:
    indexes=weather_data[weather_data[col].isnull()].index.tolist()
    print('Fixing '+str(len(indexes))+' rows in '+col)
    for i in indexes:
        x=weather_data.loc[i-3:i-1,col].sum()
        x=x+weather_data.loc[i+1:i+3,col].sum()
        weather_data.loc[i,col]=x/6
       
        

Fixing 23 rows in Temperature
Fixing 39 rows in Visibility (km)
Fixing 19 rows in Wind (km/h)
Fixing 1 rows in Precip. (mm)


In [10]:
weather_data.isnull().sum()

Unnamed: 0         0
Month              0
Day                0
Airport            0
Temperature        0
Visibility (km)    0
Wind (km/h)        0
Precip. (mm)       0
Fog                0
Hail               0
Snow               0
Thunderstorm       0
Tornado            0
dtype: int64

Saving data to 'weather2015.csv'

In [11]:
weather_data.to_csv('weather2015_final_no_missing.csv')

In [12]:
len(weather_data.Airport.unique())

322

In [27]:
scrape_2015_weather_by_airport('YAK','PAYA')

,Month,Day,Airport,Temperature,Visibility (km),Wind (km/h),Precip. (mm),Fog,Hail,Snow,Thunderstorm,Tornado
0,1,1,YAK,3,13,10,15.24,0,0,0,0,0
1,1,2,YAK,1,16,13,0.00,0,0,0,0,0
2,1,3,YAK,-2,16,27,0.00,0,0,0,0,0
3,1,4,YAK,-2,16,14,0.00,0,0,0,0,0
4,1,5,YAK,-6,16,1,0.00,0,0,0,0,0
5,1,6,YAK,-6,16,1,0.00,0,0,0,0,0
6,1,7,YAK,2,16,4,0.00,0,0,0,0,0
7,1,8,YAK,4,15,7,5.33,0,0,0,0,0
8,1,9,YAK,2,15,3,0.76,1,0,0,0,0
9,1,10,YAK,2,16,4,0,0,0,0,0,0


In [16]:
page = requests.get("https://www.wunderground.com/history/airport/PAYA/2015/1/1/CustomHistory.html?dayend=31&monthend=12&yearend=2015&req_city=&req_state=&req_statename=&reqdb.zip=&reqdb.magic=&reqdb.wmo=")
soup = BeautifulSoup(page.content, 'html.parser')
table=soup.find('table',class_="responsive obs-table daily")
c_month=0
c_day=1
tbody = table.find_all('tbody')
for i in range(len(tbody)):
    trs=tbody[i].find_all('tr')    
    if(trs[0].find('td',class_='daily-td-grey')):
        c_month=c_month+1
        c_day=1
    else:
        tds=trs[0].find_all('td')
        temp=tds[0].find_all('a')
        print(temp[0].text)
        

1
2
3
4
5
6
7
8
9
10
11
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
17
18
19
20
21
22
23
24
26
27
28
2
3
4
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
